## Linking in Spark


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/spark/deduplicate_1k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev
# !pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from splink.backends.spark import similarity_jar_location

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

In [3]:
# Disable warnings for pyspark - you don't need to include this
import warnings

spark.sparkContext.setLogLevel("ERROR")
warnings.simplefilter("ignore", UserWarning)

In [4]:
from splink import splink_datasets

pandas_df = splink_datasets.fake_1000

df = spark.createDataFrame(pandas_df)

In [5]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink import Linker, SettingsCreator, SparkAPI, block_on

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        ctl.NameComparison("first_name"),
        ctl.NameComparison("surname"),
        # ctl.date_comparison("dob", cast_strings_to_date=True),
        # TOD=Fix date comparison
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
            datetime_format="%Y%m%d",
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write BRs in their SQL form
    ],
    retain_intermediate_calculation_columns=True,
    em_convergence=0.01,
)

In [6]:
linker = Linker(df, settings, database_api=SparkAPI(spark_session=spark))
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

In [7]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e5)

In [8]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)

In [9]:
results = linker.inference.predict(threshold_match_probability=0.9)

In [10]:
results.as_pandas_dataframe(limit=5)